In [378]:
# import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import numpy as np
from scipy import stats

In [379]:
# file defines and functions
def makeOutputDir(directory, baseDir='./outputs'):
    date = datetime.datetime.now().strftime("%Y%m%d")
    output_dir = f'{baseDir}/{date}/{directory}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    return output_dir

def check_missing_values(dataset):
    return dataset.isnull().sum().to_frame()

def replace_invalid_dates(date_str):
    # Split the date string into components
    parts = date_str.split('-')
    
    # Check if the year is valid
    if len(parts) < 1 or not parts[0].isdigit() or len(parts[0]) != 4:
        return pd.NaT  # Return Not a Time if year is invalid
    
    year = parts[0]
    
    # Check if the month is valid
    if len(parts) < 2 or not parts[1].isdigit() or not (1 <= int(parts[1]) <= 12):
        month = '01'  # Default to January if month is invalid
    else:
        month = parts[1].zfill(2)  # Ensure month is two digits
    
    # Check if the day is valid
    if len(parts) < 3 or not parts[2].isdigit() or not (1 <= int(parts[2]) <= 31):
        day = '01'  # Default to the first day of the month if day is invalid
    else:
        day = parts[2].zfill(2)  # Ensure day is two digits
    
    # Construct the valid date string
    valid_date_str = f"{year}-{month}-{day}"
    
    # Convert to datetime
    return pd.to_datetime(valid_date_str)


# Constants and Variables

In [380]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# constants
plot_title = "Country Mexico"
country_name = "Mexico"

dataset_file_name = "Mexico_wosis_merged.csv"
master_dataset_file_path = f"../data/soil/{dataset_file_name}"

output_dir = makeOutputDir(country_name, "../outputs")
saveModelPath = makeOutputDir(country_name, "../models")

drop_columns = ['date', 'latitude', 'longitude', 'country_name', 'region', 'continent']

# model list
models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Linear Regression': LinearRegression(),
    'Support Vector Regressor': SVR(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor()
}

# -- do not change below this line --

---

# Load Dataset

Load master dataset

In [381]:
# load master dataset
master_dataset = pd.read_csv(master_dataset_file_path)

# replace invalid dates
master_dataset['date'] = master_dataset['date'].apply(replace_invalid_dates)

# Plot Missing Values

In [ ]:
# plot missing values
missing_values = check_missing_values(master_dataset)
sns.heatmap(missing_values, annot=True, fmt='g', cmap='Blues')
plt.title(f"Missing valus in {plot_title} dataset")
plt.show()

# Remove missing values

- Removing the columns with missing values which are more than 90% of length of dataset

In [ ]:
# remove missing columns with threshold
master_dataset = master_dataset.dropna(thresh=0.3 * len(master_dataset), axis=1)
print(f"master_dataset columns: {master_dataset.columns.tolist()}")

# Feature Engineering
Create a  new column for Bulk Density, Organic Matter based on formulaes below

Create a new column for combination of silt_plus_clay

In [ ]:
# create organic matter
master_dataset['organic_matter'] = master_dataset.apply(lambda row: 1.724 * row['orgc'], axis=1)

# create bulk density
master_dataset[f'bulk_density'] = master_dataset.apply(lambda row: 1.62-0.06 * row['organic_matter'], axis=1)

# create sum of silt plus clay
master_dataset[f'silt_plus_clay'] = master_dataset.apply(lambda row: (row['silt'] if not pd.isnull(row['silt']) else 0) + (row['clay'] if not pd.isnull(row['clay']) else 0) , axis=1)

# drop silt and clay column
master_dataset = master_dataset.drop(columns=['silt', 'clay']).copy()

print(f"\n Length of dataset: {len(master_dataset)}")

# Removing Negative Values

In [ ]:
# remove negative values ignore latitude and longitude
numeric_columns_dataset = master_dataset.select_dtypes(include=['int64', 'float64']).columns.difference(['latitude', 'longitude'])
# master_dataset = master_dataset[master_dataset[numeric_columns_dataset] >= 0]
# master_dataset = master_dataset[master_dataset[numeric_columns_dataset] >= 0]

print("Length of numeric columns:", len(numeric_columns_dataset))
print(numeric_columns_dataset)
master_dataset.head()

# Visualising distribution of data

In [ ]:
# Visualising data distribution
import math
def plot_distribution_charts(numeric_columns, dataset, title=""):
    num_cols = len(numeric_columns)

    # Define the number of rows and columns dynamically
    chart_cols = 3  # Fixed number of columns
    chart_rows = math.ceil(num_cols / chart_cols)  # Calculate required rows based on columns

    fig, axes = plt.subplots(chart_rows, chart_cols, figsize=(15, 3 * chart_rows), constrained_layout=True)
    axes = axes.flatten()  # Flatten the axes for easy iteration

    for i, col in enumerate(numeric_columns):
        sns.histplot(dataset[col], kde=True, ax=axes[i])
        axes[i].set_title(f'Distribution of {col}')

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.suptitle(f"Column distribution {title}")
    plt.show()

plot_distribution_charts(numeric_columns_dataset, master_dataset, plot_title)

# Removing Outliers using Z-Score

In [ ]:
# remove outliers for orgc and tceq
for col in numeric_columns_dataset:
    # setting upper and lower limit
    upper_limit = master_dataset[col].mean() + 3*master_dataset[col].std()
    lower_limit = master_dataset[col].mean() - 3*master_dataset[col].std()

    # count outliers
    count_outliers = master_dataset[(master_dataset[col] > upper_limit) | (master_dataset[col] < lower_limit)]

    # trim outlier using z-score
    master_dataset = master_dataset[(master_dataset[col] >= lower_limit) & (master_dataset[col] <= upper_limit)]

    if(len(count_outliers) > 0):
        print(f"Processing column {col}")
        print("Upper limit",upper_limit, "Lower limit", lower_limit)
        print("Total Outliers", len(count_outliers))

print(f"\n Length of dataset: {len(master_dataset)}")
master_dataset.head()

In [ ]:
plot_distribution_charts(numeric_columns_dataset, master_dataset, "after removing outliers")

# Replacing Missing Values with avg of the column

In [ ]:
# fill missing values with mean
master_dataset[numeric_columns_dataset] = master_dataset[numeric_columns_dataset].apply(lambda x: x.fillna(x.mean()), axis=0)
print(f"\n Length of dataset: {len(master_dataset)}")
master_dataset.head()

# Dataset Length & Missing Values

In [ ]:
# split dataset into orgc and tceq
orgc_df = master_dataset.drop(columns=['tceq']).copy()
tceq_df = master_dataset.drop(columns=['orgc']).copy()

# count data & missing values
print(f"orgc_df length: {len(orgc_df)}")
print(f"tceq_df length: {len(tceq_df)}")

# plot missing values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
orgc_missing_values = check_missing_values(orgc_df)
sns.heatmap(orgc_missing_values, annot=True, fmt='g', cmap='Blues', ax=ax1)
ax1.set_title(f"Missing valus in ORGC for {plot_title}")

tceq_missing_values = check_missing_values(tceq_df)
sns.heatmap(tceq_missing_values, annot=True, fmt='g', cmap='Blues', ax=ax2)
ax2.set_title(f"Missing valus in TCEQ for {plot_title}")
plt.suptitle(f"Missing values after splitting dataset {plot_title}")
plt.tight_layout()
plt.show()

In [391]:
# save datasets
orgc_df.to_csv(f"{output_dir}/{country_name}_wosis_orgc.csv", index=False)
tceq_df.to_csv(f"{output_dir}/{country_name}_wosis_tceq.csv", index=False)

In [ ]:
# plot line chart of orgc and tceq for each year
orgc_df['year'] = orgc_df['date'].dt.year
tceq_df['year'] = tceq_df['date'].dt.year

fig, ax = plt.subplots(2, 2, figsize=(15, 8))

sns.lineplot(data=orgc_df.groupby('year')['orgc'].count().reset_index(), x='year', y='orgc', ax=ax[0, 0])
ax[0, 0].set_title(f'Count of ORGC by Year {plot_title}')
ax[0, 0].set_xlabel('Year')
ax[0, 0].set_ylabel('Count of ORGC')

sns.lineplot(data=tceq_df.groupby('year')['tceq'].count().reset_index(), x='year', y='tceq', ax=ax[0, 1])
ax[0, 1].set_title(f'Count of TCEQ by Year {plot_title}')
ax[0, 1].set_xlabel('Year')
ax[0, 1].set_ylabel('Count of TCEQ')

sns.lineplot(data=orgc_df.groupby('year')['orgc'].mean().reset_index(), x='year', y='orgc', ax=ax[1, 0])
ax[1, 0].set_title(f'Average ORGC by Year {plot_title}')
ax[1, 0].set_xlabel('Year')
ax[1, 0].set_ylabel('Average ORGC')

sns.lineplot(data=tceq_df.groupby('year')['tceq'].mean().reset_index(), x='year', y='tceq', ax=ax[1, 1])
ax[1, 1].set_title(f'Average TCEQ by Year {plot_title}')
ax[1, 1].set_xlabel('Year')
ax[1, 1].set_ylabel('Average TCEQ')

plt.suptitle(f'Line Charts of ORGC and TCEQ by Year {plot_title}')
plt.tight_layout()
plt.show()

In [ ]:
# outliers
def plot_outliers(dataframes, title):
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    for i, dataframe in enumerate(dataframes):
        sns.boxplot(data=dataframe, ax=axes[i])
        axes[i].set_title(f'{title} {plot_title}')
        axes[i].set_xlabel('Columns')
        axes[i].set_ylabel('Values')
        axes[i].set_xticks(ticks=range(len(dataframe.columns)), labels=dataframe.columns, rotation=45)

    plt.suptitle(f'Outliers in ORGC and TCEQ datasets {plot_title}')
    plt.tight_layout()
    plt.show()


# Create copy of year column
orgc_year = orgc_df['year'].copy()
tceq_year = tceq_df['year'].copy()

# append to drop colum
drop_columns.append("year")

plot_outliers([orgc_df.drop(columns=drop_columns), tceq_df.drop(columns=drop_columns)], 'ORGC and TCEQ')

In [ ]:
# Normalize data using standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
orgc_scaled = scaler.fit_transform(orgc_df.drop(columns=drop_columns))
standard_orgc_df = pd.DataFrame(orgc_scaled, columns=orgc_df.drop(columns=drop_columns).columns)

tceq_scaled = scaler.fit_transform(tceq_df.drop(columns=drop_columns))
standard_tceq_df = pd.DataFrame(tceq_scaled, columns=tceq_df.drop(columns=drop_columns).columns)

plot_outliers([standard_orgc_df, standard_tceq_df], 'ORGC and TCEQ (after standard scaling)')

standard_orgc_df['year'] = orgc_year
standard_tceq_df['year'] = tceq_year

In [ ]:
# normalize data using min max scaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
orgc_scaled_minmax = scaler.fit_transform(orgc_df.drop(columns=drop_columns))
minmax_orgc_df = pd.DataFrame(orgc_scaled_minmax, columns=orgc_df.drop(columns=drop_columns).columns)

tceq_scaled_minmax = scaler.fit_transform(tceq_df.drop(columns=drop_columns))
minmax_tceq_df = pd.DataFrame(tceq_scaled_minmax, columns=tceq_df.drop(columns=drop_columns).columns)

plot_outliers([minmax_orgc_df, minmax_tceq_df], 'ORGC and TCEQ (after min max scaling)')

minmax_orgc_df['year'] = orgc_year
minmax_tceq_df['year'] = tceq_year

# Confussion Matrix
Illustrate the relationship between features using confussion matrix

In [ ]:
# correlation matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

corr_matrix = standard_orgc_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', ax=axes[0])
axes[0].set_title(f'ORGC {country_name} (after standard scaling)')

corr_matrix = standard_tceq_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', ax=axes[1])
axes[1].set_title(f'TCEQ {country_name} (after standard scaling)')

plt.suptitle('Correlation Matrix of ORGC and TCEQ datasets')
plt.tight_layout()
plt.show()

# Traditional Model Training & Evaluation

In [397]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

def train_model(X, y, model):
    # split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # train the model
    model.fit(X_train, y_train)
    
    # evaluate the model
    y_pred = model.predict(X_test)
    
    # Calculate regression metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return mse, mae, r2, y_test, y_pred

def validate_model(models, features, target, title, save_model=False):
    best_model = None
    best_model_score = float('inf')  # Initialize best model score as infinity

    # train the models
    fig, axes = plt.subplots(1, len(models), figsize=(20, 5))
    print(title)
    for i, (model_name, model) in enumerate(models.items()):
        mse, mae, r2, y_test, y_pred = train_model(features, target, model)

        print(f'{model_name} - MSE: {mse}, MAE: {mae}, R2: {r2}')
        # joblib.dump(model, f'{saveModelPath}{title}_{model_name}.pkl')

        # Scatter plot of predicted vs actual
        axes[i].scatter(y_test, y_pred, alpha=0.7, edgecolors='k')
        axes[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        axes[i].set_title(f'{model_name}\n(MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f})', fontsize=10)
        axes[i].set_xlabel('Actual')
        axes[i].set_ylabel('Predicted')

        # Check if this model is better than the current best model
        if mae < best_model_score:
            best_model = model
            best_model_score = mae

    # Save the best model if specified
    if save_model and best_model:
        joblib.dump(best_model, f'{saveModelPath}/{title}_best_model.pkl')
        print("Best model saved in: ", f'{saveModelPath}/{title}_best_model.pkl')

    plt.suptitle(f'Scatter plots of {title} models')
    plt.tight_layout()
    plt.figtext(0.5, 0.005,
                'The scatter plots show the predicted vs actual values for each model. '
                'The red dashed line represents the perfect prediction.', 
                ha='center', fontsize=10)
    plt.show()

# Traning model on data from year 2000 onward

In [ ]:
# Standard Scaled ORGC
orgc_year_2000 = standard_orgc_df[standard_orgc_df['year'] >= 2000].copy()
orgc_year_2000 = orgc_year_2000.drop(columns=['year'])
validate_model(models, orgc_year_2000.drop(columns=['orgc']), orgc_year_2000['orgc'], f'{country_name}_ORGC', save_model=True)

# Standard Scaled TCEQ
tceq_year_2000 = standard_tceq_df[standard_tceq_df['year'] >= 2000].copy()
tceq_year_2000 = tceq_year_2000.drop(columns=['year'])
validate_model(models, tceq_year_2000.drop(columns=['tceq']), tceq_year_2000['tceq'], f'{country_name}_TCEQ', save_model=True)

## Model Transferability to past data

In [ ]:
def plot_model_transferability(standard_df_dict, model_dict, saveModelPath, country_name):
    """
    Plot model transferability to historical data.
    
    Parameters:
    -----------
    standard_df_dict : dict
        Dictionary containing standardized dataframes for each target variable
        e.g. {'orgc': standard_orgc_df, 'tceq': standard_tceq_df}
    model_dict : dict 
        Dictionary containing trained models for each target variable
        e.g. {'orgc': orgc_model, 'tceq': tceq_model}
    saveModelPath : str
        Path where models are saved
    country_name : str
        Name of the country for which models were trained
        
    Returns:
    --------
    None. Displays plot comparing model performance on historical data.
    """
    
    fig, axes = plt.subplots(1, len(model_dict), figsize=(15, 4))
    
    for i, (target_var, model) in enumerate(model_dict.items()):
        # Get data for this target
        df = standard_df_dict[target_var]
        historical_data = df[df['year'] < 2000].copy()
        
        # Prepare features and target
        features = historical_data.drop(columns=['year', target_var])
        target = historical_data[target_var]
        
        # Make predictions
        predictions = model.predict(features)
        
        # Calculate metrics
        mse = mean_squared_error(target, predictions)
        mae = mean_absolute_error(target, predictions)
        r2 = r2_score(target, predictions)
        
        # Create scatter plot
        axes[i].scatter(target, predictions, alpha=0.7, edgecolors='k')
        axes[i].plot([target.min(), target.max()], [target.min(), target.max()], 'r--')
        axes[i].set_title(f'{target_var.upper()} Model Performance on Historical Data\n'
                         f'(MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f})')
        axes[i].set_xlabel(f'Actual {target_var.upper()}')
        axes[i].set_ylabel(f'Predicted {target_var.upper()}')

    plt.suptitle('Model Transferability to Historical Data (Pre-2000)')
    plt.tight_layout()
    plt.figtext(0.5, 0.01,
                'The scatter plots show the predicted vs actual values for historical data. '
                'The red dashed line represents the perfect prediction.',
                ha='center', fontsize=10)
    plt.show()


standard_df_dict = {
    'orgc': standard_orgc_df,
    'tceq': standard_tceq_df
}

model_dict = {
    'orgc': joblib.load(f'{saveModelPath}/{country_name}_ORGC_best_model.pkl'),
    'tceq': joblib.load(f'{saveModelPath}/{country_name}_TCEQ_best_model.pkl')
}

plot_model_transferability(standard_df_dict, model_dict, saveModelPath, country_name)

## Parameter Optimization


# Deep Learning Model Training & Evaluation


In [400]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [401]:
def split_and_scale(featurs, target, test_size=0.2, random_state=42):
    # split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(featurs, target, test_size=test_size, random_state=random_state)

    # scaling for models like SVR and KNN
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train, X_train_scaled, X_test, X_test_scaled, y_train, y_test

def build_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),  # Use Input to specify the input shape
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

def validate_dl_model(features, target, title, random_state=42, validation_split=0.2, epochs=100, batch_size=32, verbose=0):
    X_train, X_train_scaled, X_test, X_test_scaled, y_train, y_test = split_and_scale(features, target, test_size=validation_split, random_state=random_state)

    # initialise and train
    input_dim = X_train_scaled.shape[1]
    model = build_model(input_dim)
    history = model.fit(X_train_scaled, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size, verbose=verbose)

    # Evaluate on test data
    y_pred = model.predict(X_test_scaled).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Deep Learning Model - MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")
    
    # Check for overfitting/underfitting
    train_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]
    if val_loss > train_loss * 1.1:  # 10% threshold
        print("Model shows signs of overfitting (validation loss notably higher than training loss)")
    elif val_loss < train_loss * 0.9:  # 10% threshold
        print("Model shows signs of underfitting (validation loss notably lower than training loss)")
    else:
        print("Model shows good fit (validation and training losses are close)")

    # Plot training history
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f"Deep Learning Model {title} \nSplit: {validation_split} \nMSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")
    plt.show()

In [402]:
num_cols = numeric_columns_dataset.difference(['tceq', 'orgc'])

In [ ]:
various_splits = [0.2, 0.3, 0.4]

for split_num in various_splits:
    # deep learning Orgc
    validate_dl_model(orgc_df[num_cols], orgc_df['orgc'], title="ORGC", validation_split=split_num)

    # deep learning Tceq
    validate_dl_model(tceq_df[num_cols], tceq_df['tceq'], title="TCEQ", validation_split=split_num)

# Spatial Regression

### Spatial Feature Engineering